# Open Exploration

As we are interested in how dike ring 4 could best balance its economic objectives with a sustainable flood-risk management plan, we would want to first use open exploration to understand how regions in the uncertainty space map to the whole outcome map.

Using open exploration, we will systematically sample the uncertainty or decision space.

## Running the model

Importing all necessary libraries, checking that libraries are up-to-date, and choosing problem formulation.

In [18]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [19]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.0.3
2.4


In [20]:
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)
from dike_model_function import DikeNetwork  # @UnresolvedImport


def sum_over(*args):
    return sum(args)

In [21]:
from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation


ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(5)

In [22]:
#enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
for unc in dike_model.uncertainties:
    print(repr(unc))
    
uncertainties = dike_model.uncertainties

import copy
uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
CategoricalParameter('discount rate 3', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132)
RealParameter('A.1_Bmax', 30, 350)
RealParameter('A.1_pfail', 0, 1)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350)
RealParameter('A.2_pfail', 0, 1)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350)
RealParameter('A.3_pfail', 0, 1)
CategoricalParameter('A.3_Brate', [0, 1, 2])
RealParameter('A.4_Bmax', 30, 350)
RealParameter('A.4_pfail', 0, 1)
CategoricalParameter('A.4_Brate', [0, 1, 2])
RealParameter('A.5_Bmax', 30, 350)
RealParameter('A.5_pfail', 0, 1)
CategoricalParameter('A.5_Brate', [0, 1, 2])


In [23]:
#enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))
    
levers = dike_model.levers 

import copy
levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1)
IntegerParameter('0_RfR 1', 0, 1)
IntegerParameter('0_RfR 2', 0, 1)
IntegerParameter('0_RfR 3', 0, 1)
IntegerParameter('1_RfR 0', 0, 1)
IntegerParameter('1_RfR 1', 0, 1)
IntegerParameter('1_RfR 2', 0, 1)
IntegerParameter('1_RfR 3', 0, 1)
IntegerParameter('2_RfR 0', 0, 1)
IntegerParameter('2_RfR 1', 0, 1)
IntegerParameter('2_RfR 2', 0, 1)
IntegerParameter('2_RfR 3', 0, 1)
IntegerParameter('3_RfR 0', 0, 1)
IntegerParameter('3_RfR 1', 0, 1)
IntegerParameter('3_RfR 2', 0, 1)
IntegerParameter('3_RfR 3', 0, 1)
IntegerParameter('4_RfR 0', 0, 1)
IntegerParameter('4_RfR 1', 0, 1)
IntegerParameter('4_RfR 2', 0, 1)
IntegerParameter('4_RfR 3', 0, 1)
IntegerParameter('EWS_DaysToThreat', 0, 4)
IntegerParameter('A.1_DikeIncrease 0', 0, 10)
IntegerParameter('A.1_DikeIncrease 1', 0, 10)
IntegerParameter('A.1_DikeIncrease 2', 0, 10)
IntegerParameter('A.1_DikeIncrease 3', 0, 10)
IntegerParameter('A.2_DikeIncrease 0', 0, 10)
IntegerParameter('A.2_DikeIncrease 1', 0, 10)
I

In [24]:
#enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1_Expected Annual Damage 0')
ScalarOutcome('A.1_Dike Investment Costs 0')
ScalarOutcome('A.1_Expected Number of Deaths 0')
ScalarOutcome('A.2_Expected Annual Damage 0')
ScalarOutcome('A.2_Dike Investment Costs 0')
ScalarOutcome('A.2_Expected Number of Deaths 0')
ScalarOutcome('A.3_Expected Annual Damage 0')
ScalarOutcome('A.3_Dike Investment Costs 0')
ScalarOutcome('A.3_Expected Number of Deaths 0')
ScalarOutcome('A.4_Expected Annual Damage 0')
ScalarOutcome('A.4_Dike Investment Costs 0')
ScalarOutcome('A.4_Expected Number of Deaths 0')
ScalarOutcome('A.5_Expected Annual Damage 0')
ScalarOutcome('A.5_Dike Investment Costs 0')
ScalarOutcome('A.5_Expected Number of Deaths 0')
ScalarOutcome('RfR Total Costs 0')
ScalarOutcome('Expected Evacuation Costs 0')
ScalarOutcome('A.1_Expected Annual Damage 1')
ScalarOutcome('A.1_Dike Investment Costs 1')
ScalarOutcome('A.1_Expected Number of Deaths 1')
ScalarOutcome('A.2_Expected Annual Damage 1')
ScalarOutcome('A.2_Dike Investment

In [25]:
#running the model through EMA workbench
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)
 
with MultiprocessingEvaluator(dike_model) as evaluator:
    experiments,outcomes = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
[MainProcess/INFO] 20 cases completed
[MainProcess/INFO] 40 cases completed
[MainProcess/INFO] 60 cases completed
[MainProcess/INFO] 80 cases completed
[MainProcess/INFO] 100 cases completed
[MainProcess/INFO] 120 cases completed
[MainProcess/INFO] 140 cases completed
[MainProcess/INFO] 160 cases completed
[MainProcess/INFO] 180 cases completed
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [26]:
experiments

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 2,A.4_DikeIncrease 3,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,A.5_DikeIncrease 3,EWS_DaysToThreat,scenario,policy,model
0,72.0,274.605108,1,0.174721,33.330636,10,0.496791,122.593812,1,0.144774,...,1.0,8.0,10.0,1.0,2.0,7.0,3.0,50,4,dikesnet
1,69.0,346.256441,10,0.766929,46.934477,1.5,0.432543,301.458471,10,0.730824,...,1.0,8.0,10.0,1.0,2.0,7.0,3.0,51,4,dikesnet
2,45.0,77.605328,10,0.943873,227.787739,10,0.526423,194.788581,1,0.814646,...,1.0,8.0,10.0,1.0,2.0,7.0,3.0,52,4,dikesnet
3,1.0,343.474047,1.5,0.444811,144.693935,1.5,0.137951,30.872722,10,0.338987,...,1.0,8.0,10.0,1.0,2.0,7.0,3.0,53,4,dikesnet
4,5.0,264.426236,10,0.150020,306.625088,10,0.317194,100.658118,1.5,0.382357,...,1.0,8.0,10.0,1.0,2.0,7.0,3.0,54,4,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,43.0,144.149271,1.5,0.049048,154.301833,10,0.263444,197.380695,1,0.912777,...,4.0,4.0,8.0,7.0,5.0,2.0,2.0,95,7,dikesnet
196,58.0,228.608168,1.5,0.371273,58.678551,1,0.841305,349.402874,1,0.492026,...,4.0,4.0,8.0,7.0,5.0,2.0,2.0,96,7,dikesnet
197,100.0,62.867740,1,0.499979,170.194216,1,0.059787,131.941282,10,0.113828,...,4.0,4.0,8.0,7.0,5.0,2.0,2.0,97,7,dikesnet
198,17.0,329.396232,1.5,0.297648,323.132208,1.5,0.598066,245.372707,1.5,0.741488,...,4.0,4.0,8.0,7.0,5.0,2.0,2.0,98,7,dikesnet


In [27]:
outcomes

{'A.1_Expected Annual Damage 0': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'A.1_Dike Investment Costs 0': array([37255203.23710208, 37255203.237

In [28]:
import pandas as pd
outcomes_new= {k:v.tolist() for k, v in outcomes.items()}

In [29]:
outcomes_new

{'A.1_Expected Annual Damage 0': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  

In [30]:
outcome_df = pd.DataFrame.from_dict(outcomes_new)
outcome_df.head()

,A.1_Expected Annual Damage 0,A.1_Dike Investment Costs 0,A.1_Expected Number of Deaths 0,A.2_Expected Annual Damage 0,A.2_Dike Investment Costs 0,A.2_Expected Number of Deaths 0,A.3_Expected Annual Damage 0,A.3_Dike Investment Costs 0,A.3_Expected Number of Deaths 0,A.4_Expected Annual Damage 0,...,A.3_Dike Investment Costs 3,A.3_Expected Number of Deaths 3,A.4_Expected Annual Damage 3,A.4_Dike Investment Costs 3,A.4_Expected Number of Deaths 3,A.5_Expected Annual Damage 3,A.5_Dike Investment Costs 3,A.5_Expected Number of Deaths 3,RfR Total Costs 3,Expected Evacuation Costs 3
0,0.0,3.725520e+07,0.0,0.0,5.941010e+07,0.0,0.0,2.879840e+07,0.0,0.0,...,3.593766e+07,0.0,0.0,2.666556e+07,0.0,0.0,6.840036e+07,0.0,115300000.0,0.0
1,0.0,3.725520e+07,0.0,0.0,5.941010e+07,0.0,0.0,2.879840e+07,0.0,0.0,...,3.593766e+07,0.0,0.0,2.666556e+07,0.0,0.0,6.840036e+07,0.0,115300000.0,0.0
2,0.0,3.725520e+07,0.0,0.0,5.941010e+07,0.0,0.0,2.879840e+07,0.0,0.0,...,3.593766e+07,0.0,0.0,2.666556e+07,0.0,0.0,6.840036e+07,0.0,115300000.0,0.0
3,0.0,3.725520e+07,0.0,0.0,5.941010e+07,0.0,0.0,2.879840e+07,0.0,0.0,...,3.593766e+07,0.0,0.0,2.666556e+07,0.0,0.0,6.840036e+07,0.0,115300000.0,0.0
4,0.0,3.725520e+07,0.0,0.0,5.941010e+07,0.0,0.0,2.879840e+07,0.0,0.0,...,3.593766e+07,0.0,0.0,2.666556e+07,0.0,0.0,6.840036e+07,0.0,115300000.0,0.0


In [31]:
#drop outcomes that are not of interest 
outcomes_of_interest = ['A.4_Expected Annual Damage 0','A.4_Dike Investment Costs 0','A.4_Expected Number of Deaths 0',
                        'A.4_Expected Annual Damage 1','A.4_Dike Investment Costs 1','A.4_Expected Number of Deaths 1',
                        'A.4_Expected Annual Damage 2','A.4_Dike Investment Costs 2','A.4_Expected Number of Deaths 2',
                        'A.4_Expected Annual Damage 3','A.4_Dike Investment Costs 3','A.4_Expected Number of Deaths 3']

In [32]:
outcome_df = outcome_df[outcomes_of_interest]
outcome_df.head()

,A.4_Expected Annual Damage 0,A.4_Dike Investment Costs 0,A.4_Expected Number of Deaths 0,A.4_Expected Annual Damage 1,A.4_Dike Investment Costs 1,A.4_Expected Number of Deaths 1,A.4_Expected Annual Damage 2,A.4_Dike Investment Costs 2,A.4_Expected Number of Deaths 2,A.4_Expected Annual Damage 3,A.4_Dike Investment Costs 3,A.4_Expected Number of Deaths 3
0,0.0,1.610892e+07,0.0,0.0,1.646804e+07,0.0,0.0,1.011729e+07,0.0,0.0,2.666556e+07,0.0
1,0.0,1.610892e+07,0.0,0.0,1.646804e+07,0.0,0.0,1.011729e+07,0.0,0.0,2.666556e+07,0.0
2,0.0,1.610892e+07,0.0,0.0,1.646804e+07,0.0,0.0,1.011729e+07,0.0,0.0,2.666556e+07,0.0
3,0.0,1.610892e+07,0.0,0.0,1.646804e+07,0.0,0.0,1.011729e+07,0.0,0.0,2.666556e+07,0.0
4,0.0,1.610892e+07,0.0,0.0,1.646804e+07,0.0,0.0,1.011729e+07,0.0,0.0,2.666556e+07,0.0


In [36]:
outcome_df['A.4_Expected Annual Damage Total'] = outcome_df['A.4_Expected Annual Damage 0'] +outcome_df['A.4_Expected Annual Damage 1'] +outcome_df['A.4_Expected Annual Damage 2'] +outcome_df['A.4_Expected Annual Damage 3']
outcome_df['A.4_Dike Investment Costs Total'] = outcome_df['A.4_Dike Investment Costs 0']+ outcome_df['A.4_Dike Investment Costs 1']+outcome_df['A.4_Dike Investment Costs 2']+outcome_df['A.4_Dike Investment Costs 3']
outcome_df['A.4_Expected Number of Deaths Total'] = outcome_df['A.4_Expected Number of Deaths 0'] + outcome_df['A.4_Expected Number of Deaths 1']+outcome_df['A.4_Expected Number of Deaths 2'] +outcome_df['A.4_Expected Number of Deaths 3']

In [37]:
outcome_df.head()

,A.4_Expected Annual Damage 0,A.4_Dike Investment Costs 0,A.4_Expected Number of Deaths 0,A.4_Expected Annual Damage 1,A.4_Dike Investment Costs 1,A.4_Expected Number of Deaths 1,A.4_Expected Annual Damage 2,A.4_Dike Investment Costs 2,A.4_Expected Number of Deaths 2,A.4_Expected Annual Damage 3,A.4_Dike Investment Costs 3,A.4_Expected Number of Deaths 3,A.4_Expected Annual Damage Total,A.4_Dike Investment Costs Total,A.4_Expected Number of Deaths Total
0,0.0,1.610892e+07,0.0,0.0,1.646804e+07,0.0,0.0,1.011729e+07,0.0,0.0,2.666556e+07,0.0,0.0,6.935981e+07,0.0
1,0.0,1.610892e+07,0.0,0.0,1.646804e+07,0.0,0.0,1.011729e+07,0.0,0.0,2.666556e+07,0.0,0.0,6.935981e+07,0.0
2,0.0,1.610892e+07,0.0,0.0,1.646804e+07,0.0,0.0,1.011729e+07,0.0,0.0,2.666556e+07,0.0,0.0,6.935981e+07,0.0
3,0.0,1.610892e+07,0.0,0.0,1.646804e+07,0.0,0.0,1.011729e+07,0.0,0.0,2.666556e+07,0.0,0.0,6.935981e+07,0.0
4,0.0,1.610892e+07,0.0,0.0,1.646804e+07,0.0,0.0,1.011729e+07,0.0,0.0,2.666556e+07,0.0,0.0,6.935981e+07,0.0


In [ ]:
total_outcomes = outcome_df[['A.4_Expected Annual Damage Total','A.4_Expected Annual Damage Total']]

## Visual Analysis

To find out how the outcomes relate to each other, we will create a pair plot with all the total outcomes.

In [17]:
sns.pairplot(outcomes_df)

[MainProcess/INFO] no time dimension found in results


KeyboardInterrupt: 

Since we 

In [ ]:
experiments, outcomes = results
policies = experiments['policy']

data = pd.DataFrame.from_dict(outcomes)
data['policy'] = policies

sns.pairplot(data, hue='policy', vars=outcomes.keys(), diag_kind='hist')
plt.show()